In [1]:
import numpy as np
import torch


USE_CUDA = torch.cuda.is_available()
device = torch.device("cuda") if USE_CUDA else torch.device("cpu")
training = True
print(device)

cuda


TODO

- Sample (state, action)-pairs. Split them to context and target sets!
- Figure out how to split them correctly, what differences are there brtween
- Train a neural process using the HIIT library

In [2]:
from src.utils.sampler import Sampler

GRID_SIZE = 10
AGENT_VIEW_SIZE = 3
TRAJ_LENGTH = 10


sampler = Sampler(grid_size = 10, agent_view_size = 5, traj_length = 10)
batch = sampler.generate_batch([0.4,0.5])


pygame 2.5.2 (SDL 2.28.2, Python 3.10.14)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
print(f"Shape of xc: {batch[0].shape}, yc: {batch[1].shape}, xt: {batch[2].shape}, yt: {batch[3].shape}")
print(f"Device of xc: {batch[0].device}, yc: {batch[1].device}, xt: {batch[2].device}, yt: {batch[3].device}")

Shape of xc: torch.Size([1, 9, 2]), yc: torch.Size([1, 9, 5]), xt: torch.Size([1, 1, 2]), yt: torch.Size([1, 1, 5])
Device of xc: cuda:0, yc: cuda:0, xt: cuda:0, yt: cuda:0


In [25]:
from src.utils.sampler import Sampler

sampler = Sampler(grid_size = 8, agent_view_size = 3, traj_length = 20)

import numpy as np
user_params = sampler.generate_user_parameters()

traj = sampler.generate_user_trajectories(5, user_params)

print(np.array(traj))

[list([((5, 5), [1, 0, 0, 0, 0]), ((4, 5), [1, 0, 0, 0, 0]), ((3, 5), [1, 0, 0, 0, 0]), ((2, 5), [0, 0, 1, 0, 0]), ((2, 4), [0, 0, 1, 0, 0]), ((2, 3), [0, 0, 1, 0, 0]), ((2, 2), [1, 0, 0, 0, 0]), ((1, 2), [0, 1, 0, 0, 0]), ((2, 2), [0, 1, 0, 0, 0]), ((3, 2), [0, 1, 0, 0, 0]), ((4, 2), [0, 1, 0, 0, 0]), ((5, 2), [0, 1, 0, 0, 0]), ((6, 2), [0, 1, 0, 0, 0]), ((7, 2), [0, 0, 0, 1, 0]), ((7, 3), [0, 0, 0, 1, 0]), ((7, 4), [0, 0, 1, 0, 0]), ((7, 3), [0, 0, 0, 1, 0]), ((7, 4), [0, 0, 0, 1, 0]), ((7, 5), [0, 0, 0, 1, 0]), ((7, 5), [0, 0, 0, 0, 1])])
 list([((2, 3), [1, 0, 0, 0, 0]), ((1, 3), [0, 0, 0, 1, 0]), ((1, 4), [0, 0, 0, 1, 0]), ((1, 5), [0, 1, 0, 0, 0]), ((2, 5), [0, 1, 0, 0, 0]), ((3, 5), [0, 1, 0, 0, 0]), ((4, 5), [0, 1, 0, 0, 0]), ((5, 5), [0, 1, 0, 0, 0]), ((6, 5), [0, 1, 0, 0, 0]), ((7, 5), [0, 0, 1, 0, 0]), ((7, 4), [0, 0, 1, 0, 0]), ((7, 3), [1, 0, 0, 0, 0]), ((6, 3), [0, 1, 0, 0, 0]), ((7, 3), [0, 0, 1, 0, 0]), ((7, 2), [1, 0, 0, 0, 0]), ((6, 2), [1, 0, 0, 0, 0]), ((5, 2), [0, 

/tmp/ipykernel_22293/2587577198.py:10: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  print(np.array(traj))


# Conditional Neural Process

CNPs take in pairs (x, y)

### Training loop 

### Testing the library

Change: Use cross entropy

In [53]:
import neuralprocesses.torch as nps
import torch.nn.functional as F

sampler = Sampler(grid_size = 10, agent_view_size = 3, traj_length = 10)

if training:
  cnp = nps.construct_agnp(dim_x = 2, dim_y = 5, likelihood = "het", dim_embedding = 64).to("cpu")
  #cnp = nps.construct_convgnp(dim_x = 2, dim_y = 5).to("cpu")
  opt = torch.optim.Adam(cnp.parameters(), lr = 1e-4)
  total_loss = 0
  for i in range(10000):
    xc, yc, xt, yt = sampler.generate_batch([0.4,0.5])

    xc = xc.permute(0,2,1).to("cpu")
    yc = yc.permute(0,2,1).to("cpu")
    xt = xt.permute(0,2,1).to("cpu")
    yt = yt.permute(0,2,1).to("cpu")
    
    dist = cnp(xc, yc, xt)
    #mean = dist.mean.permute(0,2,1)
    #yt_indices = torch.argmax(yt, dim = -1)
    #loss = F.cross_entropy(dist.mean, yt_indices)
    
    loss = -torch.mean(nps.loglik(cnp, xc, yc, xt, yt))
    opt.zero_grad(set_to_none=True)
    loss.backward()
    opt.step()
    
    total_loss += loss.item()
    
    if i % 100 == 0:
      print(f"mean loss {total_loss/(i+1)}")
      total_loss = 0


mean loss 4.375207979079614
mean loss 4.308122355222105
mean loss 2.0004657951909284
mean loss 0.9779666955962762


KeyboardInterrupt: 

In [54]:
import os

if training:
  PATH = os.path.abspath(os.getcwd())
  torch.save(cnp.state_dict(), PATH + "/models/anp.pth")

In [55]:
if not training:
  cnp = nps.construct_agnp(dim_x = 2, dim_y = 4, likelihood = "het").to(device)
  cnp.load_state_dict(torch.load('models/anp.pth'))

cnp.eval().to("cpu")

Model(
    Chain(
        Chain(
            SqueezeParallel(),
            AssertNoParallel(),
        ),
        Copy(),
        Parallel(
            Chain(
                RepeatForAggregateInputs(
                  (coder): InputsCoder()
                ),
                DeterministicLikelihood(),
            ),
            Parallel(
                Chain(
                    RepeatForAggregateInputs(
                      (coder): Attention(
                        (encoder_x): MLP(
                          (net): Sequential(
                            (0): Linear(in_features=2, out_features=256, bias=True)
                            (1): ReLU()
                            (2): Linear(in_features=256, out_features=256, bias=True)
                            (3): ReLU()
                            (4): Linear(in_features=256, out_features=256, bias=True)
                            (5): ReLU()
                            (6): Linear(in_features=256, out_features=256, bias=True

### Make predictions on new trajectories, and compare

In [56]:
cnp.eval().to("cpu")
xc, yc, xt, yt = sampler.generate_batch()

xc = xc.permute(0,2,1).to("cpu")
yc = yc.permute(0,2,1).to("cpu")
xt = xt.permute(0,2,1).to("cpu")
yt = yt.permute(0,2,1).to("cpu")

mean, var, noiseless_samples, noisy_samples = nps.predict(cnp, xc, yc, xt)

In [7]:
xt.shape

torch.Size([25, 2, 1])

In [57]:
mean, yt

(tensor([[[0.1961],
          [0.2020],
          [0.1333],
          [0.2546],
          [0.2635]],
 
         [[0.1961],
          [0.2020],
          [0.1333],
          [0.2546],
          [0.2635]],
 
         [[0.1961],
          [0.2020],
          [0.1333],
          [0.2546],
          [0.2635]],
 
         [[0.1961],
          [0.2020],
          [0.1333],
          [0.2546],
          [0.2635]],
 
         [[0.1961],
          [0.2020],
          [0.1333],
          [0.2546],
          [0.2635]],
 
         [[0.1836],
          [0.1905],
          [0.1300],
          [0.2316],
          [0.2411]],
 
         [[0.1836],
          [0.1905],
          [0.1300],
          [0.2316],
          [0.2411]],
 
         [[0.1836],
          [0.1905],
          [0.1300],
          [0.2316],
          [0.2411]],
 
         [[0.1836],
          [0.1905],
          [0.1300],
          [0.2316],
          [0.2411]],
 
         [[0.1836],
          [0.1905],
          [0.1300],
          [0.

In [94]:
yt

tensor([[[0.],
         [0.],
         [0.],
         [0.],
         [1.]],

        [[0.],
         [0.],
         [0.],
         [0.],
         [1.]],

        [[0.],
         [0.],
         [0.],
         [0.],
         [1.]],

        [[0.],
         [0.],
         [0.],
         [0.],
         [1.]],

        [[0.],
         [0.],
         [0.],
         [0.],
         [1.]],

        [[0.],
         [0.],
         [0.],
         [0.],
         [1.]],

        [[0.],
         [0.],
         [0.],
         [0.],
         [1.]],

        [[0.],
         [0.],
         [0.],
         [0.],
         [1.]],

        [[0.],
         [0.],
         [0.],
         [0.],
         [1.]],

        [[0.],
         [0.],
         [0.],
         [0.],
         [1.]],

        [[0.],
         [0.],
         [0.],
         [0.],
         [1.]],

        [[0.],
         [0.],
         [0.],
         [0.],
         [1.]],

        [[0.],
         [0.],
         [0.],
         [0.],
         [1.]],


## The following is a copy from the relational_neural_process githu

Model

In [ ]:
import numpy as np
import numpy.random as npr
import torch
import torch.nn as nn
import torch.optim as optim
import collections
import matplotlib.pyplot as plt
import datetime
import torch.nn.functional as F

class CNPDeterministicEncoder(nn.Module):
    def __init__(self, sizes):
        super(CNPDeterministicEncoder, self).__init__()
        self.linears = nn.ModuleList()
        for i in range(len(sizes) - 1):
            self.linears.append(nn.Linear(sizes[i], sizes[i + 1]))

    def forward(self, context_x, context_y):
        """
        Encode training set as one vector representation

        Args:
            context_x: batch_size x set_size x feature_dim_x
            context_y: batch_size x set_size x feature_dim_y

        Returns: representation: batch_size x representation_size:
        """

        encoder_input = torch.cat((context_x, context_y), dim = -1)
        batch_size, set_size, filter_size = encoder_input.shape
        x = encoder_input.view(batch_size * set_size, -1)
        for i, linear in enumerate(self.linears[:-1]):
            x = torch.relu(linear(x))
        x = self.linears[-1](x)
        x = x.view(batch_size, set_size, -1)
        representation = x.sum(dim=1)
        return representation
            
class CNPDeterministicDecoder(nn.Module):
    def __init__(self, sizes):
        super(CNPDeterministicDecoder, self).__init__()
        self.linears = nn.ModuleList()
        for i in range(len(sizes) - 1):
            self.linears.append(nn.Linear(sizes[i], sizes[i + 1]))

    def forward(self, representation, target_x):
        """
        Take representation representation of current training set, and a target input x,
        return the predictive distribution at x (Gaussian with mean mu and scale sigma)

        Args:
            representation: batch_size x representation_size
            target_x: batch_size x set_size x d
        """
        batch_size, set_size, d = target_x.shape
        
        if representation is None:        
            input = target_x            
        else:
            representation = representation.unsqueeze(1).repeat([1, set_size, 1])
            input = torch.cat((representation, target_x), dim=-1)
        
        #All rows
        x = input.view(batch_size * set_size, -1)
        for linear in self.linears[:-1]:
            x = torch.relu(linear(x))
        logits = self.linears[-1](x)
        logits = logits.view(batch_size, set_size, -1)
        probs = F.softmax(logits, dim = -1)

        dist = torch.distributions.categorical.Categorical(probs = probs)
        return dist, probs, logits
    
        '''
        mu, log_sigma = torch.split(out, 1, dim = -1)
        sigma = 0.01 + 0.99 * torch.nn.functional.softplus(log_sigma)
        dist = torch.distributions.normal.Normal(loc=mu, scale=sigma)
        '''

class CNPDeterministicModel(nn.Module):
    def __init__(self, encoder_size, decoder_size):
        super(CNPDeterministicModel, self).__init__()
        self._encoder = CNPDeterministicEncoder(encoder_size)
        self._decoder = CNPDeterministicDecoder(decoder_size)


    def forward(self, query, target_y = None):
        (context_x, context_y), target_x = query
        representation = self._encoder(context_x, context_y)
        dist, probs, logits = self._decoder(representation, target_x)

        log_p = None
        if target_y is not None:
            #Reverse one hot encoding on target_y
            target_y = torch.argmax(target_y, dim = -1)
            log_p = dist.log_prob(target_y)

        return log_p, probs, logits

In [11]:
import neuralprocesses.torch as nps

d_x, d_in, representation_size, d_out, hidden_size = 2, 6, 258, 4, 128
encoder_sizes = [d_in, hidden_size, hidden_size, hidden_size, representation_size]
decoder_sizes = [representation_size + d_x, hidden_size, hidden_size, hidden_size, d_out]

model = CNPDeterministicModel(encoder_size=encoder_sizes, decoder_size=decoder_sizes)
model.to(device)

CNPDeterministicModel(
  (_encoder): CNPDeterministicEncoder(
    (linears): ModuleList(
      (0): Linear(in_features=6, out_features=128, bias=True)
      (1-2): 2 x Linear(in_features=128, out_features=128, bias=True)
      (3): Linear(in_features=128, out_features=258, bias=True)
    )
  )
  (_decoder): CNPDeterministicDecoder(
    (linears): ModuleList(
      (0): Linear(in_features=260, out_features=128, bias=True)
      (1-2): 2 x Linear(in_features=128, out_features=128, bias=True)
      (3): Linear(in_features=128, out_features=4, bias=True)
    )
  )
)